<a href="https://colab.research.google.com/github/luanakwon/PaperImplementation/blob/main/FF_AlgorithmByGHinton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised-Learning

### Simple FC structure with Backpropagation

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import MNIST

In [2]:
class SimplestLayerNorm(nn.Module):
    def __init__(self, eps=1e-05):
        super(SimplestLayerNorm, self).__init__()
        self.eps = eps

    def forward(self, x):
        norm = torch.linalg.vector_norm(x.view(x.shape[0],-1),dim=1)
        return x / (norm.view(-1,1)+self.eps)

In [13]:
class UnspvisedFCwBackProp(nn.Module):
    def __init__(self):
        super(UnspvisedFCwBackProp, self).__init__()
        self.layer0 = nn.Sequential(
            nn.Linear(1*28*28, 30),
            nn.ReLU(),
            SimplestLayerNorm()
        )
        self.layer1 = nn.Sequential(
            nn.Linear(30, 30),
            nn.ReLU(),
            SimplestLayerNorm()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(30, 30),
            nn.ReLU(),
            SimplestLayerNorm()
        )
        self.layer3 = nn.Sequential(
            nn.Linear(30, 30),
            nn.ReLU(),
            SimplestLayerNorm()
        )
        self.linearClassifier = nn.Sequential(
            nn.Linear(30*3,10),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        hiddens = []
        hiddens.append(self.layer0(x.view(x.shape[0],-1)))
        hiddens.append(self.layer1(hiddens[0]))
        hiddens.append(self.layer2(hiddens[1]))
        hiddens.append(self.layer3(hiddens[2]))
        out = self.linearClassifier(torch.cat(hiddens[1:],dim=1))
        return out

In [14]:
train_dataset = MNIST(
    root='/content',train=True,download=True,
    transform=transforms.ToTensor())
loader = torch.utils.data.DataLoader(
    dataset=train_dataset,batch_size=4,shuffle=True
)

nnet = UnspvisedFCwBackProp()

for i, (x, label) in enumerate(loader):
    print(x.shape, x.dtype, torch.mean(x))
    print(label)

    y = nnet(x)
    print(y)

    break

# x = torch.randn((2,3))
# print(x)
# print(x*x)
# print(torch.sqrt(torch.sum(x*x,dim=1)))
# y = x/torch.sqrt(torch.sum(x*x,dim=1).view(2,1))
# print(y)
# print(torch.sum(y*y,dim=1))

# # print(torch.linalg.matrix_norm(x,dim=(1,)))

# norm = SimplestLayerNorm(eps=0)
# print(norm(x))

torch.Size([4, 1, 28, 28]) torch.float32 tensor(0.1304)
tensor([2, 3, 0, 2])
tensor([[0.1074, 0.0847, 0.1155, 0.1054, 0.1063, 0.0881, 0.0897, 0.1002, 0.0982,
         0.1045],
        [0.1058, 0.0831, 0.1170, 0.1065, 0.1056, 0.0851, 0.0941, 0.1014, 0.1026,
         0.0988],
        [0.1086, 0.0881, 0.1114, 0.1047, 0.1034, 0.0910, 0.0930, 0.0994, 0.0963,
         0.1043],
        [0.1067, 0.0834, 0.1170, 0.1092, 0.1045, 0.0854, 0.0931, 0.1006, 0.0975,
         0.1025]], grad_fn=<SoftmaxBackward0>)
